In [1]:
import pandas as pd
import numpy as np
from lightfm import LightFM

## 1. Data Pre-Proceesing

### A- User Interactions

In [2]:
ratings_df = pd.read_csv("Data/Data_LightFm/ratings.csv")

In [3]:
ratings_df["userId"] = ratings_df["userId"].astype("int32")
ratings_df["movieId"] = ratings_df["movieId"].astype("int32")
ratings_df["rating"] = ratings_df["rating"].astype("float32")

In [4]:
ratings_df.head(2)

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5


In [5]:
ratings_df['rating'] = ratings_df['rating'].apply(lambda x: 1 if x >= 4 else 0)

In [6]:
ratings_df.values

array([[     1,      2,      0],
       [     1,     29,      0],
       [     1,     32,      0],
       ...,
       [138493,  69644,      0],
       [138493,  70286,      1],
       [138493,  71619,      0]])

### B- Movie Features

In [7]:
movies_df = pd.read_csv("Data/Data_LightFm/movies.csv")

In [8]:
movies_df.head(2)

,movieId,avg_movie_rating,title,movie_youth_rate,movie_popularity_rate,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,3.921240,toy story,0.83871,0.738297,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,3.211977,jumanji,0.83871,0.330446,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
movies_df.columns

Index(['movieId', 'avg_movie_rating', 'title', 'movie_youth_rate',
       'movie_popularity_rate', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [10]:
movies_df.drop(["title","avg_movie_rating","movie_youth_rate","movie_popularity_rate"], inplace=True, axis=1)

In [11]:
movies_df.head(2)

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [12]:
genres = movies_df.columns.drop("movieId")

In [13]:
movies_df["genres"]=movies_df[genres].apply(lambda row: [col for col in genres if row[col] == 1], axis=1)
movies_df.head(2)

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genres
0,1,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"[Adventure, Children, Fantasy]"


In [14]:
movie_features_matrix=list(zip(movies_df.movieId, movies_df.genres))
movie_features_matrix

[(1, ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']),
 (2, ['Adventure', 'Children', 'Fantasy']),
 (3, ['Comedy', 'Romance']),
 (4, ['Comedy', 'Drama', 'Romance']),
 (5, ['Comedy']),
 (6, ['Action', 'Crime', 'Thriller']),
 (7, ['Comedy', 'Romance']),
 (8, ['Adventure', 'Children']),
 (9, ['Action']),
 (10, ['Action', 'Adventure', 'Thriller']),
 (11, ['Comedy', 'Drama', 'Romance']),
 (12, ['Comedy', 'Horror']),
 (13, ['Adventure', 'Animation', 'Children']),
 (14, ['Drama']),
 (15, ['Action', 'Adventure', 'Romance']),
 (16, ['Crime', 'Drama']),
 (17, ['Drama', 'Romance']),
 (18, ['Comedy']),
 (19, ['Comedy']),
 (20, ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller']),
 (21, ['Comedy', 'Crime', 'Thriller']),
 (22, ['Crime', 'Drama', 'Horror', 'Mystery', 'Thriller']),
 (23, ['Action', 'Crime', 'Thriller']),
 (24, ['Drama', 'Sci-Fi']),
 (25, ['Drama', 'Romance']),
 (26, ['Drama']),
 (27, ['Children', 'Drama']),
 (28, ['Drama', 'Romance']),
 (29, ['Adventure', 'Drama', 'Fantasy'

## C. User Features

In [21]:
user_genres = ratings_df.merge(movies_df.drop(['genres'],axis=1),on="movieId", how="left").drop(["movieId",'rating'],axis=1)

In [22]:
user_genres.head(2)

,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0


In [25]:
user_genres=user_genres.groupby("userId").sum()

In [31]:
user_genres

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,
1,66,73,10,19,41,21,0,43,69,0,45,2,3,18,11,40,42,9,4
2,19,17,1,1,10,1,0,19,1,1,18,1,2,4,6,23,19,4,2
3,61,50,4,10,52,21,1,58,20,1,32,0,6,11,16,93,50,6,3
4,13,6,2,4,11,6,0,8,3,0,0,0,2,3,4,5,13,1,1
5,18,21,6,11,24,7,0,27,11,0,1,3,8,2,16,10,15,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489,8,6,2,3,11,11,0,19,5,2,2,1,0,6,5,4,11,1,1
138490,5,4,0,3,32,18,1,144,3,2,1,1,5,12,28,3,20,7,1
138491,7,8,5,6,10,3,0,4,3,0,4,2,1,0,1,4,4,0,0


In [34]:
user_features = list(zip(user_genres.index, user_genres.to_dict(orient="records")))

[(1,
  {'Action': 66,
   'Adventure': 73,
   'Animation': 10,
   'Children': 19,
   'Comedy': 41,
   'Crime': 21,
   'Documentary': 0,
   'Drama': 43,
   'Fantasy': 69,
   'Film-Noir': 0,
   'Horror': 45,
   'IMAX': 2,
   'Musical': 3,
   'Mystery': 18,
   'Romance': 11,
   'Sci-Fi': 40,
   'Thriller': 42,
   'War': 9,
   'Western': 4}),
 (2,
  {'Action': 19,
   'Adventure': 17,
   'Animation': 1,
   'Children': 1,
   'Comedy': 10,
   'Crime': 1,
   'Documentary': 0,
   'Drama': 19,
   'Fantasy': 1,
   'Film-Noir': 1,
   'Horror': 18,
   'IMAX': 1,
   'Musical': 2,
   'Mystery': 4,
   'Romance': 6,
   'Sci-Fi': 23,
   'Thriller': 19,
   'War': 4,
   'Western': 2}),
 (3,
  {'Action': 61,
   'Adventure': 50,
   'Animation': 4,
   'Children': 10,
   'Comedy': 52,
   'Crime': 21,
   'Documentary': 1,
   'Drama': 58,
   'Fantasy': 20,
   'Film-Noir': 1,
   'Horror': 32,
   'IMAX': 0,
   'Musical': 6,
   'Mystery': 11,
   'Romance': 16,
   'Sci-Fi': 93,
   'Thriller': 50,
   'War': 6,
   'We

---

## 2. Dataset preparation

In [525]:
from lightfm.data import Dataset
dataset = Dataset()

In [526]:
dataset.fit(
    users=ratings_df["userId"],
    items=ratings_df["movieId"],
    item_features=genres,
    user_features=genres,
)

In [527]:
dataset.model_dimensions()

(138493, 26708)

In [528]:
dataset.item_features_shape()

(26689, 26708)

In [ ]:
dataset.user_features_shape()

### A- Interactions

In [529]:
interactions, weights = dataset.build_interactions(ratings_df.values)

In [532]:
from lightfm.cross_validation import random_train_test_split

train_interactions, test_interactions = random_train_test_split(
    interactions, test_percentage=0.7, random_state=42
)
train_weights, test_weights = random_train_test_split(
    weights, test_percentage=0.7, random_state=42
)

In [533]:
train_interactions, test_interactions

(<138493x26689 sparse matrix of type '<class 'numpy.int32'>'
 	with 5999824 stored elements in COOrdinate format>,
 <138493x26689 sparse matrix of type '<class 'numpy.int32'>'
 	with 13999591 stored elements in COOrdinate format>)

In [534]:
train_weights, test_weights

(<138493x26689 sparse matrix of type '<class 'numpy.float32'>'
 	with 5999824 stored elements in COOrdinate format>,
 <138493x26689 sparse matrix of type '<class 'numpy.float32'>'
 	with 13999591 stored elements in COOrdinate format>)

### B- Movies Features

In [535]:
processed_movie_features = dataset.build_item_features(movie_features_matrix,normalize=False)

## C- User Features

In [ ]:
processed_user_features = dataset.build_user_features(user_features)

---

## 3. Model

In [536]:
model = LightFM(loss="warp", learning_schedule="adagrad", random_state=42)
model.fit(
    interactions=train_interactions,
    sample_weight=train_weights,
    item_features=processed_movie_features,
    user_features=processed_user_features,
    epochs=50,
    num_threads=14,
    verbose=True,
)

Epoch: 100%|██████████| 50/50 [03:04<00:00,  3.69s/it]


---

## 4. Evaulation

In [298]:
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score

train_roc_auc = auc_score(
    model,
    train_interactions,
    item_features=processed_movie_features,
    user_features=processed_user_features,
    num_threads=14,
).mean()
test_roc_auc = auc_score(
    model,
    test_interactions,
    item_features=processed_movie_features,
    user_features=processed_user_features,
    num_threads=14,
).mean()

train_precision = precision_at_k(
    model,
    train_interactions,
    item_features=processed_movie_features,
    user_features=processed_user_features,
    num_threads=14,
).mean()
test_precision = precision_at_k(
    model,
    test_interactions,
    train_interactions=train_interactions,
    item_features=processed_movie_features,
    user_features=processed_user_features,
    k=15,
    num_threads=14,
).mean()

train_recall = recall_at_k(
    model,
    train_interactions,
    item_features=processed_movie_features,
    user_features=processed_user_features,
    num_threads=14,
).mean()
test_recall = recall_at_k(
    model,
    test_interactions,
    train_interactions=train_interactions,
    k=15,
    item_features=processed_movie_features,
    user_features=processed_user_features,
    num_threads=14,
).mean()

In [299]:
print(f"Train ROC AUC: {train_roc_auc:.2f}\tTest ROC AUC: {test_roc_auc:.2f}")
print(f"Train Precision: {train_precision:.2f}\tTest Precision: {test_precision:.2f}")
print(f"Train Recall: {train_recall:.2f}\tTest Recall: {test_recall:.2f}")

Train ROC AUC: 0.98	Test ROC AUC: 0.97
Train Precision: 0.18	Test Precision: 0.43
Train Recall: 0.08	Test Recall: 0.11


---

## Cross Validation

In [303]:
import numpy as np
import scipy.sparse as sp

def _shuffle(uids, iids, data, random_state):
    shuffle_indices = np.arange(len(uids))
    random_state.shuffle(shuffle_indices)
    return uids[shuffle_indices], iids[shuffle_indices], data[shuffle_indices]

def inverse_cross_validation_split(interactions, n_splits=5, random_state=None):
    """
    Split interactions into `n_splits` test sets and `n-1` train sets for inverse cross-validation.
    
    This function creates `n_splits` folds of the dataset, with each fold being used as the training set,
    and the remaining `n_splits-1` folds combined to form the test set.

    Parameters
    ----------
    interactions: scipy sparse matrix
        The interactions to split.
    n_splits: int, optional
        Number of folds. Must be at least 2.
    random_state: int or numpy.random.RandomState, optional
        Random seed used to initialize the numpy.random.RandomState number generator.
        Accepts an instance of numpy.random.RandomState for backwards compatibility.

    Yields
    ------
    (train, test): (scipy.sparse.COOMatrix, scipy.sparse.COOMatrix)
        A generator yielding `n_splits` tuples of (train data, test data).
    """
    
    if not sp.issparse(interactions):
        raise ValueError("Interactions must be a scipy.sparse matrix.")
    
    if not isinstance(random_state, np.random.RandomState):
        random_state = np.random.RandomState(seed=random_state)
    
    interactions = interactions.tocoo()
    shape = interactions.shape
    uids, iids, data = interactions.row, interactions.col, interactions.data
    
    uids, iids, data = _shuffle(uids, iids, data, random_state)
    
    fold_size = len(uids) // n_splits
    
    for i in range(n_splits):
        train_start = i * fold_size
        train_end = (i + 1) * fold_size if i < n_splits - 1 else len(uids)
        
        train_idx = slice(train_start, train_end)
        test_idx = np.concatenate([np.arange(0, train_start), np.arange(train_end, len(uids))])
        
        train = sp.coo_matrix(
            (data[train_idx], (uids[train_idx], iids[train_idx])),
            shape=shape,
            dtype=interactions.dtype,
        )
        test = sp.coo_matrix(
            (data[test_idx], (uids[test_idx], iids[test_idx])),
            shape=shape,
            dtype=interactions.dtype,
        )
        
        yield train, test


In [304]:
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score

def test_model(model,train_interactions,test_interactions,processed_movie_features,processed_user_features):
    train_roc_auc = auc_score(
        model,
        train_interactions,
        item_features=processed_movie_features,
        user_features=processed_user_features,
        num_threads=14,
    ).mean()
    test_roc_auc = auc_score(
        model,
        test_interactions,
        item_features=processed_movie_features,
        user_features=processed_user_features,
        num_threads=14,
    ).mean()

    train_precision = precision_at_k(
        model,
        train_interactions,
        item_features=processed_movie_features,
        user_features=processed_user_features,
        num_threads=14,
    ).mean()
    test_precision = precision_at_k(
        model,
        test_interactions,
        train_interactions=train_interactions,
        item_features=processed_movie_features,
        user_features=processed_user_features,
        k=15,
        num_threads=14,
    ).mean()

    train_recall = recall_at_k(
        model,
        train_interactions,
        item_features=processed_movie_features,
        user_features=processed_user_features,
        num_threads=14,
    ).mean()
    test_recall = recall_at_k(
        model,
        test_interactions,
        train_interactions=train_interactions,
        k=15,
        item_features=processed_movie_features,
        user_features=processed_user_features,
        num_threads=14,
    ).mean()
    print(f"Train ROC AUC: {train_roc_auc:.2f}\tTest ROC AUC: {test_roc_auc:.2f}")
    print(f"Train Precision: {train_precision:.2f}\tTest Precision: {test_precision:.2f}")
    print(f"Train Recall: {train_recall:.2f}\tTest Recall: {test_recall:.2f}")
    return train_roc_auc,test_roc_auc,train_precision,test_precision,train_recall,test_recall

In [305]:
from scipy.sparse import coo_matrix
# Assuming you have a sparse interaction matrix named 'interactions'
n_splits = 5
random_state = 42
results={}
model = LightFM(loss="warp", learning_schedule="adagrad", random_state=random_state)
for i, (train, test) in enumerate(inverse_cross_validation_split(interactions, n_splits=n_splits, random_state=random_state)):
    print(f"Fold {i + 1}")
    model.fit(
        train,
        epochs=50,
        num_threads=14,
        verbose=True,
        item_features=processed_movie_features,
        user_features=processed_user_features,
    )
    results[i+1]=test_model(model,train,test,processed_movie_features,processed_user_features)
    print("Train set shape:", train.shape, "Test set shape:", test.shape)


Fold 1


Epoch: 100%|██████████| 50/50 [00:35<00:00,  1.41it/s]


Train ROC AUC: 0.99	Test ROC AUC: 0.98
Train Precision: 0.12	Test Precision: 0.47
Train Recall: 0.09	Test Recall: 0.11
Train set shape: (138493, 26689) Test set shape: (138493, 26689)
Fold 2


Epoch: 100%|██████████| 50/50 [00:36<00:00,  1.37it/s]


Train ROC AUC: 0.99	Test ROC AUC: 0.98
Train Precision: 0.13	Test Precision: 0.47
Train Recall: 0.09	Test Recall: 0.11
Train set shape: (138493, 26689) Test set shape: (138493, 26689)
Fold 3


...

Train ROC AUC: 0.99	Test ROC AUC: 0.98
Train Precision: 0.12	Test Precision: 0.46
Train Recall: 0.09	Test Recall: 0.11
Train set shape: (138493, 26689) Test set shape: (138493, 26689)
Fold 4


Epoch: 100%|██████████| 50/50 [00:36<00:00,  1.36it/s]


Train ROC AUC: 0.99	Test ROC AUC: 0.98
Train Precision: 0.12	Test Precision: 0.46
Train Recall: 0.09	Test Recall: 0.11
Train set shape: (138493, 26689) Test set shape: (138493, 26689)
Fold 5


Epoch: 100%|██████████| 50/50 [00:44<00:00,  1.11it/s]


Train ROC AUC: 0.99	Test ROC AUC: 0.98
Train Precision: 0.12	Test Precision: 0.46
Train Recall: 0.09	Test Recall: 0.11
Train set shape: (138493, 26689) Test set shape: (138493, 26689)


---

## Prediction

In [537]:
items_mapping = {v: k for k, v in dataset.mapping()[2].items()}
items_mapping

{0: 2,
 1: 29,
 2: 32,
 3: 47,
 4: 50,
 5: 112,
 6: 151,
 7: 223,
 8: 253,
 9: 260,
 10: 293,
 11: 296,
 12: 318,
 13: 337,
 14: 367,
 15: 541,
 16: 589,
 17: 593,
 18: 653,
 19: 919,
 20: 924,
 21: 1009,
 22: 1036,
 23: 1079,
 24: 1080,
 25: 1089,
 26: 1090,
 27: 1097,
 28: 1136,
 29: 1193,
 30: 1196,
 31: 1198,
 32: 1200,
 33: 1201,
 34: 1208,
 35: 1214,
 36: 1215,
 37: 1217,
 38: 1219,
 39: 1222,
 40: 1240,
 41: 1243,
 42: 1246,
 43: 1249,
 44: 1258,
 45: 1259,
 46: 1261,
 47: 1262,
 48: 1266,
 49: 1278,
 50: 1291,
 51: 1304,
 52: 1321,
 53: 1333,
 54: 1348,
 55: 1350,
 56: 1358,
 57: 1370,
 58: 1374,
 59: 1387,
 60: 1525,
 61: 1584,
 62: 1750,
 63: 1848,
 64: 1920,
 65: 1967,
 66: 1994,
 67: 1997,
 68: 2021,
 69: 2100,
 70: 2118,
 71: 2138,
 72: 2140,
 73: 2143,
 74: 2173,
 75: 2174,
 76: 2193,
 77: 2194,
 78: 2253,
 79: 2288,
 80: 2291,
 81: 2542,
 82: 2628,
 83: 2644,
 84: 2648,
 85: 2664,
 86: 2683,
 87: 2692,
 88: 2716,
 89: 2761,
 90: 2762,
 91: 2804,
 92: 2872,
 93: 2918,
 94

In [538]:
original_ratings = pd.read_csv("Data/Data_LightFm/ratings.csv")

In [541]:
movies_df = pd.read_csv("Data/archive/movie.csv")

In [542]:
def make_pred (user_id, model,movies_df,nb_preds,processed_movie_features,processed_user_features):
    n_items = dataset.item_features_shape()[0]
    mapped_user_id = dataset.mapping()[0][user_id]
    watched_movies = ratings_df[ratings_df["userId"]==user_id]["movieId"].values
    mapped_watched_movies = [dataset.mapping()[2][movie] for movie in watched_movies]
    non_watched_movies = np.setdiff1d(np.arange(n_items), mapped_watched_movies)
    prediction = model.predict(user_ids=mapped_user_id,item_ids=non_watched_movies,item_features=processed_movie_features,user_features=processed_user_features)
    predicted_movies = prediction.argsort()[::-1] [:nb_preds]
    results = [items_mapping[item] for item in predicted_movies]
    recommended_movies = pd.DataFrame({"movieId":results,"title":movies_df[movies_df["movieId"].isin(results)]["title"],"genres":movies_df[movies_df["movieId"].isin(results)]["genres"],"Score":sorted(prediction)[::-1][:nb_preds]})
    return recommended_movies

In [543]:
def get_watched_movies(user_id):
    mapped_user_id = dataset.mapping()[0][user_id]
    watched_movies = interactions.tocsr()[mapped_user_id]
    watched_movies.indices
    mapped_watched_movies = [items_mapping[movie_id] for movie_id in watched_movies.indices]
    rated_watched_movies = pd.DataFrame({"movieId":mapped_watched_movies,"title":movies_df[movies_df.movieId.isin(mapped_watched_movies)].title}).merge(ratings_df[ratings_df.userId==user_id],on="movieId",how="left")
    return original_ratings[original_ratings.userId==user_id].merge(movies_df,on="movieId",how="left").sort_values("rating",ascending=False).set_index("movieId").loc[rated_watched_movies.sort_values("rating",ascending=False)["movieId"]]

In [544]:
def user_pred(user_id,nb_preds=10):
    print("Watched Movies by user ",user_id)
    display(get_watched_movies(user_id)[:nb_preds])
    print("Recommended Movies for user ",user_id)
    display(make_pred(user_id,model,movies_df,nb_preds))

In [547]:
user_pred(89471,nb_preds=10)

Watched Movies by user  89471


,userId,rating,title,genres
movieId,,,,
106920,89471,4.0,Her (2013),Drama|Romance|Sci-Fi
1258,89471,4.5,"Shining, The (1980)",Horror
126245,89471,4.5,Homework (1989),(no genres listed)
87004,89471,5.0,Pina (2011),Documentary|Musical
90430,89471,4.0,Carnage (2011),Comedy|Drama
8951,89471,4.0,Vera Drake (2004),Drama
7178,89471,4.0,"Great Gatsby, The (1974)",Drama
2691,89471,4.0,"Legend of 1900, The (a.k.a. The Legend of the ...",Drama
89759,89471,5.0,"Separation, A (Jodaeiye Nader az Simin) (2011)",Drama


Recommended Movies for user  89471


,movieId,title,genres,Score
31,593,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,4.785706
46,50,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,3.154094
49,296,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,3.045170
257,1219,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,2.793283
293,47,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,2.770668
347,541,"Corrina, Corrina (1994)",Comedy|Drama|Romance,2.751662
537,260,Blade Runner (1982),Action|Sci-Fi|Thriller,2.739196
587,351,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,2.707761
1193,32,Psycho (1960),Crime|Horror,2.635305
2578,2664,Invasion of the Body Snatchers (1956),Horror|Sci-Fi|Thriller,2.579049
